# Object oriented programming

Object Oriented Programming (OOP) is a programming *paradigm* that allows us to
create custom types with their own properties and/or behaviors. The paradigm
allows us to organize data and operations that act on that data by particular
concepts. When done right, OOP can help make our code more comprehendible,
better organized and more flexible. When done wrong it can turn our code into an
all out mess!

To code in an object oriented way requires programming language support. Python
provides this.

In [ ]:
from typing import Self, Optional, Callable
from datetime import date
import random

## The 3 pillars of OOP

- **Encapsulation**: keep things together and hide what's not relevant
- **Inheritance**: reuse common functionality
- **Polymorphism**: objects of common super type are substitutable (*changing
  shape*)

### Desire for encapsulation

Say we have three separate strings for first, middle and last name. We can
always just organize this with three separate variables.

Could also have 3 separate variables:

In [ ]:
first_name = "First" 
middle_name = "Middle" 
last_name = "last"

However, since it's all a part of the same thing, it's convenient to keep these
things together.

In [ ]:
name = ("First", "Middle", "Last")
name

However, tuples are immutable. We can't change them.

In [ ]:
name[0] = "Change name?"

Lists perhaps give too much freedom.

In [ ]:
name = ["First", "Middle", "Last"]

In [ ]:
name[0] = "Ben"
name.append("Bean")
name.append("Benny")
name

Also, which index represented first, middle and last? Not necessarily clear from
the ordering. Class attributes or properties that are clearly defined will help
us solve this issue.

### Desire for inheritance

Say we're building a systematic trading system and we want to be able to program
different strategies. Some behaviors and properties that might go along with a
strategy could be the following:

**Strategy interface:**

- `starting_cash`: the initial amount of capital allocated to the strategy
- `portfolio`: the strategy's holdings
- `submit_order(...)`: submit an order to a broker
- `load_historical_data(...)`: load historical data over some look back period
- `on_market_data(...)`: how the strategy responds to a new market data event
- `on_order_update(...)`: how the strategy responds when there's an update to an
  order it submitted

Which of these might be common across all strategies and which might change with
each strategy? Inheritance allows us to inherit common functionality from a
parent type and override parts of an interface for details specific to the child
type.

### Desire for polymorphism

In the example above, Polymorphism would allow us to achieve more code reuse and
flexibility by composing the `Strategy` type we create with other families of
types, such as a `DataProvider` or a `Broker` type.

- We might have data stored (a `DiskDataProvider`) or in a database (a
  `DatabaseDataProvider`) and the particular strategy wouldn't care as long as
  the providers conform to the same interface
- We could have a broker we use for live trading (a `LiveBroker`), and one we
  use for backtesting, e.g. (a `BacktestBroker`). Similarly, these can be
  swapped out in the `Strategy` to allow us to reuse the strategy's code for
  both live trading and testing.

We've seen also seen Polymorphism in action by passing around functions with the
same signature and why this is useful. All univariate functions were
substitutable.

> In Python we don't *need* objects to be of the same parent type to achieve
> polymorphism, like we would in a language such as C#, since Python is a
> duck-typed language. However it can be helpful for documentation purposes.
> (See below for details on duck typing)

## Basics

### Creating a class

A *class* is a name for our custom type. We can define what data it will store,
what properties it will have, and what operations it will have.

The most basic thing we can do.

In [ ]:
class Name:
    pass

From our class, we can create multiple *instances* (objects).

In [ ]:
name1 = Name()
name2 = Name()
name1 is name2

### Dynamic attributes

Can be attached to objects.

In [ ]:
name1.first = "Ben"
name1.last = "Carter"
print(name1.first, name1.last)

However it is at the object level...

In [ ]:
print(name2.first, name2.last)

We want to define some properties our class can take.

One way is with *class variables* which have a default value that all instances
of the class will have, but can be overwritten by particular instances.

In [ ]:
class Name:
    # A couple class level variables with defaults
    first: str = ""
    last: str = ""

In [ ]:
name = Name()

In [ ]:
name.first, name.last

In [ ]:
name.first = "Tiger"
name.last = "Woods"

In [ ]:
name.first, name.last

In [ ]:
name2 = Name()
name2.first, name2.last

Now we've changed the default for all other instances.

In [ ]:
Name.first = "Phil"

In [ ]:
name2.first

Class variables are good when you want to share common traits, but for things
like a name that are expected to be different across instances, *instance*
variables are better.

### Constructors

Constructors are special *methods* that are called when an instance of an object
is created. Recall that *methods* are functions that operate on a class or
instance of a class. It can be used to initialize state. 

While this whole process occurs in one method in most languages, in Python
there's two of them to do this job:

- `__new__`: called first to construct the object which takes care of memory
  allocation
- `__init__`: called second to initialize the object

You can override these methods as needed.

> NOTE: you will probably never need to override `__new__`, but it's very common
> to override `__init__`.


#### `__init__`

Set up just like a normal function, but notice the `self` parameter. This always
shows up first on *instance methods* which are methods that act on an instance
of an class.

In [ ]:
class Name:
    def __init__(self) -> None:
        self.first: str = ""
        self.last: str = ""

In [ ]:
name1 = Name()
name2 = Name()
print("name1:", name1.first, name1.last)
print("name2:", name2.first, name2.last)
name1 is name2

In [ ]:
name1.first = "Ivan"
name1.last = "Oder"
print("name1:", name1.first, name1.last)
print("name2:", name2.first, name2.last)

Since `__init__` is just like a regular function, arguments can be provided the
same way (after `self`).

In [ ]:
class Name:
    def __init__(
        self, first: str, last: str, middle: str = "", nickname: Optional[str] = None
    ) -> None:
        self.first = first
        self.last = last
        self.middle = middle
        self.nickname = nickname or "Old sport"  # default nickname

Now we can pass in state during the construction / initialization process.

In [ ]:
name1 = Name("Ivan", "Oder")
print(name1.first, name1.last)

In [ ]:
name2 = Name("Earl", "Riser", middle="Lee")
print(name2.first, name2.middle, name2.last)

#### `__new__`

So what does new do exactly? Maybe the one thing you might do with `__new__` is
use it to create *singleton* types, which is a design pattern which restricts
all objects of types to one instance.

In [ ]:
class Singleton:
    # Class variable that holds an instance to itself 
    _obj: Self = None # type: ignore

    def __new__(cls) -> Self:
        # We only create a new instance if we haven't already done so
        if cls._obj is None:
            cls._obj = super().__new__(cls)
        return cls._obj

    def __init__(self) -> None:
        # We can still have instance data
        self.data = "Some data"

In [ ]:
s1 = Singleton()
s2 = Singleton()

These are the same object.

In [ ]:
s1 is s2

In [ ]:
s1.data, s2.data

In [ ]:
s1.data = "Some other data"

In [ ]:
s1.data, s2.data

##### When would you use this?

### Instance methods

`__init__` was one instance method, however we can define others that are
indented under the class name with first argument of `self`.

In [ ]:
from typing import Optional
class Name:
    def __init__(self, first: str, last: str, middle: Optional[str] = None) -> None:
        self.first = first
        self.last = last
        self.middle = middle

    def has_middle_name(self) -> bool:
        """Whether has a middle name"""
        return self.middle is not None
    
    def full_name(self) -> str:
        """Full name string representation"""
        if self.has_middle_name():
            return f"{self.first} {self.middle} {self.last}"
        else:
            return f"{self.first} {self.last}"


In [ ]:
name1 = Name("Ivan", "Oder")
name2 = Name("Earl", "Riser", middle="Lee")

In [ ]:
name1.has_middle_name(), name2.has_middle_name()

In [ ]:
name1.full_name(), name2.full_name()

### Other dunder instance methods

#### `__repr__`

- Used for debugging, and is invoked for REPL output or if called `repr()`
- Often designed to give a way to reconstruct the object from a string using `eval()`

In [ ]:
class NameNoRepr:
    def __init__(self, first: str, last: str) -> None:
        self.first = first
        self.last = last

In [ ]:
name_no_repr = NameNoRepr("Bart", "Ender")
name_no_repr, str(name_no_repr)

In [ ]:
class NameWithRepr:
    def __init__(self, first: str, last: str) -> None:
        self.first = first
        self.last = last
    
    def __repr__(self) -> str:
        return f"NameWithRepr(first='{self.first}', last='{self.last}')"

In [ ]:
name_with_repr = NameWithRepr("Bart", "Ender")
name_with_repr # Wil print automatically

In [ ]:
name_with_repr_str = repr(name_with_repr)
name_with_repr_str

In [ ]:
recreate = eval(name_with_repr_str)
type(recreate), recreate

`__repr__` will be used to generate a string from the object if no `__str__`
method is defined.

In [ ]:
repr(name_with_repr), str(name_with_repr)

#### `__str__`

- Used to create a more readable representation, i.e. how you'd want to display
  if printing it

In [ ]:
class NameWithReprAndStr:
    def __init__(self, first: str, last: str) -> None:
        self.first = first
        self.last = last
    
    def __repr__(self) -> str:
        return f"NameWithRepr(first='{self.first}', last='{self.last}')"
        
    def __str__(self) -> str:
        return f"{self.first} {self.last}"

In [ ]:
name_with_repr_and_str = NameWithReprAndStr("Constance", "Noring")

In [ ]:
name_with_repr_and_str # invokes repr() implicitly

In [ ]:
print(name_with_repr_and_str) # invokes str() implicitly

Unambiguously

In [ ]:
repr(name_with_repr_and_str), str(name_with_repr_and_str)

#### `__call__`

Can be used to make an instance of a class a *callable*. Let's drop the name
example for now.

In [ ]:
class MyFunc:
    def __call__(self) -> None:
        print("Function was called")

In [ ]:
f = MyFunc()
f()

This can be useful for many things. Previously, we created univariate
parameterized distribution functions using closures.

In [ ]:
def make_normal_pdf(mu: float, sigma: float) -> Callable[[float], float]:
    def wrapper(x: float) -> float:
        print("Normal with:", mu, sigma)
        return  1.2345 # Normal PDF implementation

    return wrapper

In [ ]:
n1 = make_normal_pdf(5, 10)
result = n1(4.2)

Now we can do this by defining a class.

In [ ]:
from typing import Any


class NormalPdf:
    def __init__(self, mu: float, sigma: float) -> None:
        self.mu = mu
        self.sigma = sigma

    def __call__(self, x: float) -> Any:
        print("Normal with:", self.mu, self.sigma)
        return  1.2345 # Normal PDF implementation

In [ ]:
n1 = NormalPdf(5, 10)
result = n1(4.2)

##### Exercise

How would you create a function call tracker to track the number of times a
function was called and with and what arguments? This can be a useful thing to
do if you're having trouble understanding the behavior of your optimizer.

In [ ]:
from typing import Callable, Iterable

class UnivariateCallTracker:
    """A function counter that works for our univariate function"""

    def __init__(self, func: Callable[[float], float]) -> None:
        """Pass in a function to the counter"""
        # Fill in details
    
    def __call__(self, val: float) -> float:
        """Call implementation"""
        # Fill in details

    def call_count(self) -> int:
        """Return number of times function was called"""
        # Fill in details

    def call_values(self) -> Iterable[float]:
        """Call count invocations"""
        # Fill in details

    def reset(self) -> None:
        """Reset the counter"""
        # Fill in details


### Class attributes

Can be assigned outside of any instance methods and are accessible to all
classes of that type. Used for attributes that should generally always be the
same across instances.

Example, all humans have brains!

In [ ]:
class Name:
    def __init__(self, first: str, last: str) -> None:
        self.first = first
        self.last = last

    def __str__(self) -> str:
        return f"{self.first} {self.last}"


class Person:
    HAS_A_BRAIN: str = "Yes!"

    def __init__(self, name: Name) -> None:
        self.name = name

    def __str__(self) -> str:
        return f"{self.name} has a brain? {self.HAS_A_BRAIN}"

In [ ]:
barbara_liskov = Person(Name("Barbara", "Liskov"))
print(barbara_liskov)

In [ ]:
mahatma_ghandi = Person(Name("Mahatma", "Ghandi"))
print(mahatma_ghandi)

In [ ]:
fischer_black = Person(Name("Fischer", "Black"))
print(fischer_black)

In [ ]:
ben_carter = Person(Name("Ben", "Carter"))
print(ben_carter)

If you really need to, you can override a class variable on a particular
instance.

In [ ]:
ben_carter.HAS_A_BRAIN = "Debatable 🙃"
print(ben_carter)

In [ ]:
print(barbara_liskov)
print(mahatma_ghandi)
print(fischer_black)

### Class methods

Methods can also be defined at the class level. These are often used as a more
self-documenting `__init__` method.

You need the `classmethod` decorator.

In [ ]:
from typing import Self


class Name:
    def __init__(self, first: str, last: str) -> None:
        self.first = first
        self.last = last

    @classmethod
    def from_csv(cls, csv: str) -> Self:
        first, last = csv.split(",")
        return cls(first, last)

    def __str__(self) -> str:
        return f"{self.first} {self.last}"

In [ ]:
name1 = Name("Barb", "Dwyer")
name2 = Name.from_csv("Ella,Vader")
name3 = Name.from_csv("Sue,Yu")
print(name1)
print(name2)
print(name3)

An example from the wild.

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame.from_dict({"a": [1, 2, 3], "b": ["d", "e", "f"]})

### Inheritance

This is the mechanism by which we can reuse common functionality.

In [ ]:
class Creature:
    """Parent class for living creatures"""

    def __init__(self, name: str = "") -> None:
        self.name = name

    def print_name(self):
        print(self.name)

    def eat(self, food: str) -> None:
        print(f"Yum, {self.name} is eating {food} with gratitude")


class Person(Creature):
    def __init__(self, name: str) -> None:
        super().__init__(name)

    def talk(self) -> None:
        print("I'd like a coffee")


class Dog(Creature):
    def __init__(self, name: str) -> None:
        super().__init__(name)

    def bark(self) -> None:
        print("Woof grr bark woof")


class Cat(Creature):
    def __init__(self, name: str) -> None:
        super().__init__(name)

    def meow(self) -> None:
        print("Woof grr bark woof")


class SuperMan(Creature):
    def __init__(self, name: str) -> None:
        super().__init__(name)

    def fly(self) -> None:
        print(self.name, "is flying")


class NotSuperMan(Creature):
    def __init__(self, name: str) -> None:
        # No super() call here
        self.name

    def fly(self) -> None:
        print(self.name, "cannot flying")

Let's create some objects.

In [ ]:
me = Person("Ben")
my_cat1 = Cat("Mac")
my_cat2 = Cat("Basil")
my_cat3 = Cat("Bean Joe")
my_dog1 = Dog("Holly")
my_dog2 = Dog("Noodle")

We're all subtypes of the same thing.

In [ ]:
isinstance(me, Creature), isinstance(my_cat1, Creature)

We can invoke common functionality defined in the parent class.

In [ ]:
me.eat("gruel")

In [ ]:
my_cat1.eat("cat food")

In [ ]:
my_dog1.eat("garbage")

In [ ]:
me.print_name()
my_dog1.print_name()

Also we have our own more specific methods that are only relevant for the
subtype.

In [ ]:
me.talk()
my_dog1.bark()

> The calls to `super().__init__()` to initialize the parent class. This is
> important to properly initialize the state held in the parent.

In [ ]:
super_man = SuperMan("Clark")
super_man.print_name()


In [ ]:
ben = NotSuperMan("Ben")
ben.print_name()

#### Overriding methods

Sometimes we want to have default functionality but specialize it for certain
instances. For example, we had different methods to vocalize above *bark*,
*talk*, etc.

In [ ]:
class Name:
    def __init__(self, first: str, last: str) -> None:
        self.first = first
        self.last = last

    def __str__(self) -> str:
        return f"{self.first} {self.last}"


class SpeakableCreature:
    def __init__(self, name: Name) -> None:
        self.name = name

    def print_name(self) -> None:
        print(self.name)

    def speak(self) -> None:
        """By default, a speakable creature will make a general groaning sound"""
        print("Myaaeeeeuuu")


class Zombie(SpeakableCreature):
    def __init__(self, name: Name) -> None:
        super().__init__(name)


class Person(SpeakableCreature):
    def __init__(self, name: Name) -> None:
        super().__init__(name)

    def speak(self) -> None:
        print("Rock on man!")


class Dog(SpeakableCreature):
    def __init__(self, name: Name) -> None:
        super().__init__(name)

    def speak(self) -> None:
        print("Woof grr bark woof")

class Baby(SpeakableCreature):
    def __init__(self, name: Name) -> None:
        super().__init__(name)

In [ ]:
living_rob = Person(Name("Rob", "Downey Jr."))
zombie_rob = Zombie(Name("Rob", "Zombie"))
dog_rob = Dog(Name("Rob", "The dog"))
baby_rob = Baby(Name("Rob", "The baby"))

In [ ]:
living_rob.speak() # Uses overridden
zombie_rob.speak() # Uses default
dog_rob.speak() # Uses overridden
baby_rob.speak() # Uses default

## Basic concepts and usage

### Inheriting from Python exceptions

A common practice is if one of the built-in exception types is not relevant, for
your use case then inherit from the most relevant one to create your specific
one.

In [ ]:
class BeingLazyException(RuntimeError):
    def __init__(self, msg: str) -> None:
        super().__init__(msg)

In [ ]:
raise BeingLazyException("It's the weekend! I'm having a cocktail, come back later.")

### Hiding things

Generally it's good practice to show as little about the class as possible and
make things that users of the type don't need to know about *private* (this is
part of encapsulation).

In Python there's no way to actually do this, so we rely on convention of
leading underscore.

In [ ]:
class MoodyCalculation:
    """Performs a calculation based off its mood"""
    def __init__(self, val: float) -> None:
        self.val = val # We're ok letting this get accessed
        self._weekend_days = (5, 6) # An implementation detail that's private

    def perform_calculation(self) -> None:
        today = date.today()
        if self._is_weekend(today):
            raise BeingLazyException("I don't want to do this on a weekend!")
        elif self._is_tired(today):
            raise BeingLazyException("I'm le tired")
        self.val = 42 * today.weekday()

    def _is_weekend(self, d: date) -> bool:
        """Private helper method to determine if weekend"""
        return d.weekday() in self._weekend_days

    def _is_tired(self, d: date) -> bool:
        tired_day = random.randint(0, 6)
        tired = False
        if d.weekday() == tired_day:
            tired = True
        return tired
            

Keep running this.

In [ ]:
calculation = MoodyCalculation(1234)
calculation.perform_calculation()
calculation.val

### Enums

Act as labels for a collection of things.

In [ ]:
from enum import IntEnum

class Weekday(IntEnum):
    MONDAY = 1
    TUESDAY = 2
    WEDNESDAY = 3
    THURSDAY = 4
    FRIDAY = 5
    SATURDAY = 6
    SUNDAY = 7

In [ ]:
class_day = Weekday.WEDNESDAY

In [ ]:
if class_day == Weekday.WEDNESDAY:
    print("FM 5151 is tonight!")

In [ ]:
class_day # Repr

In [ ]:
class_day.value, class_day.name

### Polymorphism

Objects can be substituted that share the same interface. Above we defined
different creatures. They're all different subclasses of `SpeakableCreature`
but can be substituted for each other because of their common interface.


In [ ]:
creatures = [dog_rob, living_rob, zombie_rob]
def print_creatures(creatures: list[SpeakableCreature]):
    for creature in creatures:
        print("---")
        creature.print_name()
        creature.speak()
        print("---")

print_creatures(creatures)

#### Duck typing

Note that since Python is a *duck typed* language, we don't have to have the
same common type passed in the loop above. C# is an example of a language where
we would.

*If it walks like a duck and talks like a duck then it must be a duck*

In [ ]:
class Duck:
    def __init__(self, first: str, last: str) -> None:
        self.first = first
        self.last = last

    def print_name(self):
        print(f"{self.first} {self.last}".upper())

    def speak(self):
        print("QUACK")

In [ ]:
creatures.append(Duck("Donald", "Duck"))
print_creatures(creatures)